In [1]:
import importlib

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from apollo.settings import LONG_SIGNAL, NO_SIGNAL, SHORT_SIGNAL
from apollo.utils.common import to_default_date_string

In [2]:
# Manually reloading python module such that
# jupyter reflects changes without kernel restart

import apollo.api.yahoo_api_connector as yac
import apollo.calculations.average_true_range as atr
import apollo.calculations.chaikin_accumulation_distribution as cad
import apollo.calculations.hull_moving_average as hma
import apollo.calculations.keltner_channel as kc

importlib.reload(yac)
importlib.reload(hma)
importlib.reload(kc)
importlib.reload(atr)
importlib.reload(cad);

In [3]:
ticker = "SPY"
start_date = "2023-06-01"
end_date = "2024-01-01"

api_connector = yac.YahooApiConnector(ticker, start_date, end_date)
dataframe = api_connector.request_or_read_prices()

dataframe;

In [4]:
WINDOW_SIZE = 20

hma_calculator = hma.HullMovingAverageCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)
hma_calculator.calculate_hull_moving_average()

atr_calculator = atr.AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)
atr_calculator.calculate_average_true_range()

kc_calculator = kc.KeltnerChannelCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
    volatility_multiplier=1.1,
)
kc_calculator.calculate_keltner_channel()

cad_calculator = cad.ChaikinAccumulationDistributionCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)
cad_calculator.calculate_chaikin_accumulation_distribution_line()

dataframe.dropna(inplace=True)

dataframe["signal"] = NO_SIGNAL

long = (dataframe["adj close"] > dataframe["lkc_bound"]) & (
    dataframe["adl"] > dataframe["prev_adl"]
)
dataframe.loc[long, "signal"] = LONG_SIGNAL

short = (dataframe["adj close"] < dataframe["ukc_bound"]) & (
    dataframe["adl"] < dataframe["prev_adl"]
)
dataframe.loc[short, "signal"] = SHORT_SIGNAL

dataframe

,ticker,open,high,low,close,adj close,volume,hma,tr,atr,lkc_bound,ukc_bound,adl,prev_adl,signal
date,,,,,,,,,,,,,,,
2023-07-27,SPY,459.019989,459.440002,451.549988,452.489990,447.687775,92194400,452.087304,7.890015,3.387054,448.361545,455.813063,1.209467e+08,2.143889e+08,-1
2023-07-28,SPY,455.880005,457.779999,452.489990,456.920013,452.070770,80011800,452.243799,5.290009,3.482201,448.413377,456.074220,1.154855e+08,1.209467e+08,-1
2023-07-31,SPY,457.410004,458.160004,456.049988,457.790009,452.931549,62040400,452.518266,2.110016,3.413592,448.763315,456.273217,1.179430e+08,1.154855e+08,1
2023-08-01,SPY,456.269989,457.250000,455.489990,456.480011,451.635468,55291500,452.718485,2.300018,3.357913,449.024780,456.412190,1.051785e+08,1.179430e+08,-1
2023-08-02,SPY,453.250000,453.519989,449.350006,450.130005,445.352844,93933400,452.199940,7.130005,3.546518,448.298770,456.101109,3.258856e+07,1.051785e+08,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,SPY,473.859985,475.380005,471.700012,473.649994,472.182892,67126600,473.633471,3.679993,4.328476,468.872147,478.394795,4.332101e+08,4.434859e+08,-1
2023-12-26,SPY,474.070007,476.579987,473.989990,475.649994,474.176697,55387000,474.636393,2.929993,4.258552,469.951986,479.320800,4.706698e+08,4.332101e+08,1
2023-12-27,SPY,475.440002,476.660004,474.890015,476.510010,475.034058,68000300,475.594958,1.769989,4.134124,471.047422,480.142494,5.251270e+08,4.706698e+08,1


In [5]:
fig = make_subplots(rows=1, cols=1, specs=[[{}]], vertical_spacing=0)

# Plot closing prices
fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["adj close"],
        line={"color": "blue", "width": 1},
        name="Adj close",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["lkc_bound"],
        line={"color": "red", "width": 1},
        name="Lower KC",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["ukc_bound"],
        line={"color": "red", "width": 1},
        name="Upper KC",
    ),
    row=1,
    col=1,
)

fig.update_layout(
    title={"text": f"{ticker}", "x": 0.5},
    height=650,
    plot_bgcolor="#EFF5F8",
)

fig.update_yaxes(visible=False, secondary_y=True)

In [6]:
prime_value = "close"
trace_value = "adl"

x = dataframe.index.to_numpy()

y1 = dataframe[prime_value].to_numpy()
y2 = dataframe[trace_value].to_numpy()

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name=prime_value)

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis="y2")

# Plot title
title = (
    f"{ticker}:"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)